In [ ]:
%%script bash
mkdir scrapers/
mkdir scrapers/amazon
cd scrapers/amazon
python -m pip install virtualenv
virtualenv env
. env/bin/activate
python -m pip install scrapy
python -m pip install numpy
python -m pip install pandas
python -m pip install requests
python -m pip install scraperapi-sdk
scrapy startproject amazon_scraper

In [ ]:
%%script bash
python -m pip install pandas

In [ ]:
import pandas as pd
data = pd.read_csv('store_credentials .csv', header=0, sep=';', index_col=False)
data

In [ ]:
%%writefile settings.py

In [ ]:
%%script bash
mv settings.py scrapers/amazon/amazon_scraper/amazon_scraper/

In [ ]:
%%writefile amazon_spider.py
import scrapy
import numpy as np
import pandas as pd
#from scraper_api import ScraperAPIClient

class AmazonSpider(scrapy.Spider):
    name = 'amazon'
    client = ScraperAPIClient('yourAPI')
    start_urls = [client.scrapyGet(url='https://yandex.ru/search/?text=СЕО%20оптимизация%20сайта%20СЕО%20Сайта%20СЕО%20продвижение%20СЕО%20оптимизация%20и%20продвижение&lr=213'),]
    search_engine='https://yandex.ru'
    url_data = np.array([])
    url_counter = 0

    def parse(self, response):
        links =\
        response.css('div.organic__path a::attr(href)').getall()
        self.url_data = np.append(self.url_data, np.array(links))
        self.url_counter+=len(links)     
        next_page = response.css('a.pager__item_kind_next::attr(href)').get()
        if (self.url_counter<2500) and (next_page is not None):
            yield scrapy.Request(self.client.scrapyGet(url = self.search_engine+next_page), self.parse)
        
    def closed(self, reason):
        print(self.url_counter)
        raw = pd.DataFrame(data=self.url_data)
        raw.to_csv(r'/home/jovyan/raw_url.csv', index = False, header=False)

In [ ]:
%%script bash 
mv amazon_spider.py scrapers/amazon/amazon_scraper/amazon_scraper/spiders

In [ ]:
%%script bash
cd scrapers/amazon
. env/bin/activate
cd amazon_scraper/
scrapy crawl amazon